In [2]:
import pickle
import glob

In [4]:
def get_data_from_pickle(x):
    with open(x, 'rb') as f:
        result = pickle.load(f)
    return result

In [5]:
tag_matrix = get_data_from_pickle('tag_matrix.pickle')

In [6]:
tag_numbering = get_data_from_pickle('tag_numbering.pickle')

In [9]:
import pandas as pd

In [11]:
df = pd.read_json('data_version_2.json')

In [12]:
tag_list = [tag for tags in df.tags for tag in tags]

In [14]:
unique_tag_list = list(set(tag_list))

In [15]:
!pip install gensim
from gensim.models import Word2Vec, FastText

In [16]:
corpus = df.tags.values

In [17]:
corpus

array([list(['까페', '잔잔한']),
       list(['운동', '드라이브', 'Pop', '트로피컬하우스', '힐링', '기분전환', '2017', '팝', '트렌드', '일렉']),
       list(['잔잔한', '추억', '회상']), ..., list(['인디']), list(['여친']),
       list(['설렘', '사랑'])], dtype=object)

In [30]:
# CBOW(sg=0) : 주어진 단어에 앞뒤로 몇개의 단어를 Input으로 사용하여 주어진 단어를 유추하는 모델
# skip-gram(sg=1) : 현재 단어 하나를 가지고 주위에 등장하는 몇개의 단어들의 등장 유무를 유추


Type:           Word2Vec
String form:    Word2Vec(vocab=1382, size=10, alpha=0.025)
File:           ~/opt/anaconda3/lib/python3.7/site-packages/gensim/models/word2vec.py
Docstring:     
Train, use and evaluate neural networks described in https://code.google.com/p/word2vec/.

Once you're finished training a model (=no more updates, only querying)
store and use only the :class:`~gensim.models.keyedvectors.KeyedVectors` instance in `self.wv` to reduce memory.

The model can be stored/loaded via its :meth:`~gensim.models.word2vec.Word2Vec.save` and
:meth:`~gensim.models.word2vec.Word2Vec.load` methods.

The trained word vectors can also be stored/loaded from a format compatible with the
original word2vec implementation via `self.wv.save_word2vec_format`
and :meth:`gensim.models.keyedvectors.KeyedVectors.load_word2vec_format`.

Some important attributes are the following:

Attributes
----------
wv : :class:`~gensim.models.keyedvectors.Word2VecKeyedVectors`
    This object essentially conta

In [18]:
#임베딩(w2v_skipgram)모델 생성
%time Skip_Gram_model = Word2Vec(corpus, size=10, window=10, min_count=1,  workers=8, sg=1, iter=10)

CPU times: user 2.05 s, sys: 19.2 ms, total: 2.07 s
Wall time: 798 ms


In [19]:
Skip_Gram_model.most_similar('여행', topn=10)

/Users/joyoungmin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('휴양지', 0.8958439826965332),
 ('휴일', 0.8623803853988647),
 ('해변', 0.8599976301193237),
 ('기분전환', 0.8586515188217163),
 ('드라이브', 0.8563899993896484),
 ('운전', 0.8547537326812744),
 ('비행기', 0.8532578945159912),
 ('바캉스', 0.8526214361190796),
 ('여름', 0.8491854667663574),
 ('바다', 0.8478224277496338)]

In [20]:
Skip_Gram_model.most_similar('성탄절', topn=10)

/Users/joyoungmin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('크리스마스캐롤', 0.9929973483085632),
 ('메리크리스마스', 0.9926170110702515),
 ('크리스마스노래', 0.9914136528968811),
 ('Christmas', 0.988018274307251),
 ('캐럴', 0.9838935136795044),
 ('크리스마스이브', 0.9802675247192383),
 ('캐롤', 0.9665442109107971),
 ('크리스마스', 0.945396900177002),
 ('12월', 0.9253143072128296),
 ('겨울노래', 0.9234572649002075)]

In [21]:
tag_vectors = []
for tag in unique_tag_list:
    tag_vectors.append(Skip_Gram_model[tag])

/Users/joyoungmin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [48]:
tag_vectors[1381]

array([-0.19348139,  0.56510955, -0.43994173,  0.41321546, -0.8927941 ,
       -0.25548512, -0.14549626,  0.2310634 , -0.2999666 ,  0.793347  ],
      dtype=float32)

- 모델 인퍼런스 


In [28]:
from sklearn.cluster import KMeans

In [51]:
# kmeans2 = KMeans(n_clusters=2, random_state=0).fit(tag_vectors)
# kmeans4 = KMeans(n_clusters=4, random_state=0).fit(tag_vectors)
# kmeans6 = KMeans(n_clusters=6, random_state=0).fit(tag_vectors)
# kmeans8 = KMeans(n_clusters=8, random_state=0).fit(tag_vectors)
kmeans10 = KMeans(n_clusters=10, random_state=0).fit(tag_vectors)
# kmeans12 = KMeans(n_clusters=12, random_state=0).fit(tag_vectors)
# kmeans14 = KMeans(n_clusters=14, random_state=0).fit(tag_vectors)
# kmeans16 = KMeans(n_clusters=16, random_state=0).fit(tag_vectors)
# kmeans18 = KMeans(n_clusters=18, random_state=0).fit(tag_vectors)
# kmeans20 = KMeans(n_clusters=20, random_state=0).fit(tag_vectors)
kmeans15 = KMeans(n_clusters=15, random_state=0).fit(tag_vectors)

In [52]:
from collections import defaultdict

In [53]:
tag_dict = defaultdict(list)
for tag, label in zip(unique_tag_list, kmeans15.labels_):
    tag_dict[label].append(tag)
for idx in tag_dict.keys():
    print(idx, len(tag_dict[idx]),"\n", tag_dict[idx][:12])    
# for i in range(6):
#     print(i, tag_dict[i][:10])

6 65 
 ['결혼식', '따스함', '고막남친', '여친', '연애', '커플', '연인', '달달한', '고백송', '달달한노래', '봄바람', '설렘']
4 286 
 ['Queen', '칠아웃', '위클리초이스', '편곡', '졸릴때', 'J_Rock', '팝뮤직', '남녀듀엣', '남성보컬', '시부야케이', '전통가요', '전곡듣기']
10 70 
 ['electronica', '내적댄스', '디스코', '딥하우스', '러닝머신', 'Club', '이디엠', '일렉트로니카', '웨이트', '사운드', '흥폭발', '파티']
3 173 
 ['간지', '한국힙합', '팝송', '댄스음악', '1980', '광고', '시상식', '덥스텝', '웹진웨이브', '파워워킹', '멋진', '딘']
0 76 
 ['지친하루', '노동요', '직장인', '힐링음악', '그냥', '위로가필요할때', '월요병', '청춘', '부드러운_잔잔한', '버스', '지친', '설날']
13 101 
 ['방콕', '더위', '시원한', '기차', '경쾌', '자전거', '비행기', '봄나들이', '공원', '귀여운', '한강', '여름']
1 99 
 ['싱그러운', '선물', '팝재즈', '잔잔한음악', '재즈', '조용', '카페음악', '강아지', '배경음악', '커피', '어쿠스틱팝', '쌀쌀']
14 101 
 ['음색깡패', '라운지', '스윗마인드', 'drive', '알앤비소울', '빈티지', '리듬감', '끈적', '매장음악', '장르불문', 'Trendy', '그루브']
9 23 
 ['메리크리스마스', '눈오는날', '겨울밤', '쌀쌀한', '크리스마스', '크리스마스노래', '겨울노래', '캐롤', '추위', '12월', '겨울감성', '추운']
12 41 
 ['사색', '잠', '연주곡', '치유', '낮잠', '뉴에이지', '편안한', '책읽을때', '공부할때', '아기', '어린이집', '자장가']
2 63 
 ['레드벨벳', '성인가요', '회

In [54]:
for i in range(15):
    print(i, len(tag_dict[i]),"\n",tag_dict[i][:12])

0 76 
 ['지친하루', '노동요', '직장인', '힐링음악', '그냥', '위로가필요할때', '월요병', '청춘', '부드러운_잔잔한', '버스', '지친', '설날']
1 99 
 ['싱그러운', '선물', '팝재즈', '잔잔한음악', '재즈', '조용', '카페음악', '강아지', '배경음악', '커피', '어쿠스틱팝', '쌀쌀']
2 63 
 ['레드벨벳', '성인가요', '회식', '영탁', '남자아이돌', '내마음의사진', '아이돌숨은명곡', '2019년', '아이돌그룹', '방탄소년단', '에이핑크', '아이돌의숨은명곡']
3 173 
 ['간지', '한국힙합', '팝송', '댄스음악', '1980', '광고', '시상식', '덥스텝', '웹진웨이브', '파워워킹', '멋진', '딘']
4 286 
 ['Queen', '칠아웃', '위클리초이스', '편곡', '졸릴때', 'J_Rock', '팝뮤직', '남녀듀엣', '남성보컬', '시부야케이', '전통가요', '전곡듣기']
5 93 
 ['락', '한국', 'Soundtrack', '가요', '사랑설렘', '가사로듣는', '감동', '장르', '숨은명곡', '드라마ost', '애니메이션', '여자가수']
6 65 
 ['결혼식', '따스함', '고막남친', '여친', '연애', '커플', '연인', '달달한', '고백송', '달달한노래', '봄바람', '설렘']
7 39 
 ['홀로', '헤어짐', '이별노래', '권태기', '눈물', '가사', '솔로', '기억', '아련', '상처', '보고싶다', '슬픔']
8 111 
 ['감성적인', '우울', '고요한', '잠자기전', '새벽', '감성인디', '감성곡', '목소리', '비올때', '우울한날', '마무리', '선선한']
9 23 
 ['메리크리스마스', '눈오는날', '겨울밤', '쌀쌀한', '크리스마스', '크리스마스노래', '겨울노래', '캐롤', '추위', '12월', '겨울감성', '추운']
10 70 
 ['electronic

In [55]:
tag_dict_numerical = tag_dict

In [56]:
for i in range(15):
    for j in range(len(tag_dict_numerical[i])):
        tag_dict_numerical[i][j] = tag_numbering[tag_dict_numerical[i][j]]